In [1]:
import pandas as pd
import numpy as np

In [2]:
mouse_file = "data/Mouse_metadata.csv"
mouse_df = pd.read_csv(mouse_file, encoding="ISO-8859-1")
mouse_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16
...,...,...,...,...,...
244,z314,Stelasyn,Female,21,28
245,z435,Propriva,Female,12,26
246,z581,Infubinol,Female,24,25
247,z795,Naftisol,Female,13,29


In [ ]:
study_file = "data/Study_results.csv"
study_df = pd.read_csv(study_file, encoding="ISO-8859-1")
study_df